# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [381]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal # To be able to convert string field from csv file into decimal. 
from cassandra.query import named_tuple_factory
from cassandra.query import dict_factory # To work with Panda dataframe.

#### Creating list of filepaths to process original event csv data files

In [382]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [383]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [384]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [385]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [386]:
# I have chosen the Keyspace name to be the same as the startup's!
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [387]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Answers:

### A1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [388]:
# Drop table "musicAppHistory_session_itemInSession" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_session_itemInSession"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)

In [389]:
# Create table "musicAppHistory_session_itemInSession".
# The Partition Key is "sessionId" since it forms the core object of the table based on which it gets distributed.
# The Primary Key is made up of the Partition Key "sessionID" and the Clustering Column "itemInSession" since this
## composite key makes the Primary Key unique.
create_table_query = "CREATE TABLE IF NOT EXISTS musicAppHistory_session_itemInSession"
create_table_query = create_table_query + "(sessionId int, itemInSession int, artistName text, songName text, songLength decimal, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

In [390]:
# Open local file "event_datafile_new.csv".
file = 'event_datafile_new.csv'

# Read line by line from the file and insert the selected fields from the line into the table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)# Read line by line from the file.
    next(csvreader) # skip header.
    for line in csvreader:
        
#insert the selected fields from the line into table "musicAppHistory_session_itemInSession".
        insert_into_query = "INSERT INTO musicAppHistory_session_itemInSession (sessionId, itemInSession, artistName, songName, songLength)"
        insert_into_query  = insert_into_query + "VALUES (%s, %s, %s, %s, %s)"
        # Notice the inline data type conversion from string to integer and decimal.
        try:
            session.execute(insert_into_query, (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5])))
        except Exception as e:
            print(e)


In [391]:
# Run select statement to provide answer.
query = "SELECT * FROM musicAppHistory_session_itemInSession WHERE sessionId=338 AND itemInSession=4"

try:
    session.row_factory = named_tuple_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#     
for row in rows:
    # Notice that the column names returned in "rows" are automatically changed to all lower case (e.g. sessionId
    ## gives an error)
    print (">> ", "Session ID: ", row.sessionid, " || ", "Item in Session: ", row.iteminsession, " || ", "Artist Name: ", row.artistname, " || ", "Song Name: ", row.songname, " || ", "Song Length: ", row.songlength, " <<")

>>  Session ID:  338  ||  Item in Session:  4  ||  Artist Name:  Faithless  ||  Song Name:  Music Matters (Mark Knight Dub)  ||  Song Length:  495.3073  <<


In [392]:
# Run select statement to provide answer with Panda (ref https://groups.google.com/a/lists.datastax.com/g/python-driver-user/c/1v-KHtyA0Zs).
query = "SELECT * FROM musicAppHistory_session_itemInSession WHERE sessionId=338 AND itemInSession=4"
try:
    session.row_factory = dict_factory
    df = pd.DataFrame()
    for row in session.execute(query):
        df = pd.DataFrame(row, index=[0])
        print(df)
    df = df.reset_index(drop=True).fillna(pd.np.nan)
    
except Exception as e:
    print(e)

   sessionid  iteminsession artistname songlength  \
0        338              4  Faithless   495.3073   

                          songname  
0  Music Matters (Mark Knight Dub)  


### A2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [393]:
# Drop table "musicAppHistory_user_session_itemInSession" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_user_session_itemInSession"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)              

In [394]:
# Create table "musicAppHistory_user_session_itemInSession".
# The Partition Key is "userId" since it forms the core object of the table based on which it gets distributed.
# The Primary Key is made up of the Partition Key "userId" and the Clustering Columns "sessionId" and "itemInSession" 
## since this composite key makes the Primary Key unique.
create_table_query = "CREATE TABLE IF NOT EXISTS musicAppHistory_user_session_itemInSession"
create_table_query = create_table_query + "(userId int, sessionId int, itemInSession int, artistName text, songName text, userFirstName text, userLastName text, PRIMARY KEY (userId, sessionId, itemInSession))"

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

In [395]:
# Open local file "event_datafile_new.csv".
file = 'event_datafile_new.csv'

# Read line by line from the file and insert the selected fields from the line into the table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)# Read line by line from the file.
    next(csvreader) # skip header.
    for line in csvreader:
        
#insert the selected fields from the line into table "musicAppHistory_user_session_itemInSession".
        insert_into_query = "INSERT INTO musicAppHistory_user_session_itemInSession (userId, sessionId, itemInSession, artistName, songName, userFirstName, userLastName)"
        insert_into_query  = insert_into_query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Notice the inline data type conversion from string to integer.
        try:
            session.execute(insert_into_query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)


In [396]:
# Run select statement to provide answer.
query = "SELECT * FROM musicAppHistory_user_session_itemInSession WHERE userId=10 AND sessionId=182"

try:
    session.row_factory = named_tuple_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#     
for row in rows:
    # Notice that the column names returned in "rows" are automatically changed to all lower case (e.g. sessionId
    ## gives an error)
    print (">> ", "User ID: ", row.userid, " || ", "Session ID: ", row.sessionid, " || ", "Item in Session: ", row.iteminsession, " || ", "Artist Name: ", row.artistname, " || ", "Song Name: ", row.songname, " || ", "User First Name: ", row.userfirstname, " || ", "User Last Name:", row.userlastname, " <<")

>>  User ID:  10  ||  Session ID:  182  ||  Item in Session:  0  ||  Artist Name:  Down To The Bone  ||  Song Name:  Keep On Keepin' On  ||  User First Name:  Sylvie  ||  User Last Name: Cruz  <<
>>  User ID:  10  ||  Session ID:  182  ||  Item in Session:  1  ||  Artist Name:  Three Drives  ||  Song Name:  Greece 2000  ||  User First Name:  Sylvie  ||  User Last Name: Cruz  <<
>>  User ID:  10  ||  Session ID:  182  ||  Item in Session:  2  ||  Artist Name:  Sebastien Tellier  ||  Song Name:  Kilometer  ||  User First Name:  Sylvie  ||  User Last Name: Cruz  <<
>>  User ID:  10  ||  Session ID:  182  ||  Item in Session:  3  ||  Artist Name:  Lonnie Gordon  ||  Song Name:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  ||  User First Name:  Sylvie  ||  User Last Name: Cruz  <<


In [397]:
# Run select statement to provide answer with Panda (ref https://groups.google.com/a/lists.datastax.com/g/python-driver-user/c/1v-KHtyA0Zs).
query = "SELECT * FROM musicAppHistory_user_session_itemInSession WHERE userId=10 AND sessionId=182"
try:
    session.row_factory = dict_factory
    df = pd.DataFrame()
    for row in session.execute(query):
        df = pd.DataFrame(row, index=[0])
        print(df)
    df = df.reset_index(drop=True).fillna(pd.np.nan)
    
except Exception as e:
    print(e)

   userid  sessionid  iteminsession        artistname            songname  \
0      10        182              0  Down To The Bone  Keep On Keepin' On   

  userfirstname userlastname  
0        Sylvie         Cruz  
   userid  sessionid  iteminsession    artistname     songname userfirstname  \
0      10        182              1  Three Drives  Greece 2000        Sylvie   

  userlastname  
0         Cruz  
   userid  sessionid  iteminsession         artistname   songname  \
0      10        182              2  Sebastien Tellier  Kilometer   

  userfirstname userlastname  
0        Sylvie         Cruz  
   userid  sessionid  iteminsession     artistname  \
0      10        182              3  Lonnie Gordon   

                                            songname userfirstname  \
0  Catch You Baby (Steve Pitron & Max Sanna Radio...        Sylvie   

  userlastname  
0         Cruz  


### A3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [398]:
# Drop table musicAppHistory_songName_userFirstName_userLastName" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_songName_userName"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)              

In [399]:
# Create table "musicAppHistory_songName_userName".
# The Partition Key is "songName" since it forms the core object of the table based on which it gets distributed.
# The Primary Key is made up of the Partition Key "songName" and the Clustering Columns "userFirstName" and 
## "itemInSession". 
# While it is true that in the csv file there are duplicate rows formed from the composite fields "songName" and 
## "userFirstName" and "userLastNames", Cassandra uses an UPSERT (any existing row with a matching composite columns 
## value gets overwritten by the new inserted row), meaning that eventually we end up with a unique row in the table. 
create_table_query = "CREATE TABLE IF NOT EXISTS musicAppHistory_songName_userName"
create_table_query = create_table_query + "(songName text, userFirstName text, userLastName text, PRIMARY KEY (songName, userFirstName, userLastName))"

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

In [400]:
# Open local file "event_datafile_new.csv".
file = 'event_datafile_new.csv'

# Read line by line from the file and insert the selected fields from the line into the table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)# Read line by line from the file.
    next(csvreader) # skip header.
    for line in csvreader:
        
#insert the selected fields from the line into table "musicAppHistory_songName_userName".
        insert_into_query = "INSERT INTO musicAppHistory_songName_userName (songName, userFirstName, userLastName)"
        insert_into_query  = insert_into_query + "VALUES (%s, %s, %s)"
        # Notice the inline data type conversion from string to integer.
        try:
            session.execute(insert_into_query, (line[9], line[1], line[4]))
        except Exception as e:
            print(e)


In [401]:
# Run select statement to provide answer.
query = "SELECT * FROM musicAppHistory_songName_userName WHERE songName='All Hands Against His Own'"

try:
    session.row_factory = named_tuple_factory
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#     
for row in rows:
    print (">> ", "Song Name: ", row.songname, " || ", "User First Name: ", row.userfirstname, " || ", "User Last Name:", row.userlastname, " <<")

>>  Song Name:  All Hands Against His Own  ||  User First Name:  Jacqueline  ||  User Last Name: Lynch  <<
>>  Song Name:  All Hands Against His Own  ||  User First Name:  Sara  ||  User Last Name: Johnson  <<
>>  Song Name:  All Hands Against His Own  ||  User First Name:  Tegan  ||  User Last Name: Levine  <<


In [402]:
# Run select statement to provide answer with Panda (ref https://groups.google.com/a/lists.datastax.com/g/python-driver-user/c/1v-KHtyA0Zs).
query = "SELECT * FROM musicAppHistory_songName_userName WHERE songName='All Hands Against His Own'"  
try:
    session.row_factory = dict_factory
    df = pd.DataFrame()
    for row in session.execute(query):
        df = pd.DataFrame(row, index=[0])
        print(df)
    df = df.reset_index(drop=True).fillna(pd.np.nan)
    
except Exception as e:
    print(e)

                    songname userfirstname userlastname
0  All Hands Against His Own    Jacqueline        Lynch
                    songname userfirstname userlastname
0  All Hands Against His Own          Sara      Johnson
                    songname userfirstname userlastname
0  All Hands Against His Own         Tegan       Levine


### Drop the tables before closing out the sessions

In [403]:
# Drop table "musicAppHistory_session_itemInSession" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_session_itemInSession"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)

In [404]:
# Drop table "musicAppHistory_user_session_itemInSession" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_user_session_itemInSession"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)              

In [405]:
# Drop table musicAppHistory_songName_userFirstName_userLastName" if it already exists.
drop_table_if_exists = "DROP TABLE IF EXISTS musicAppHistory_songName_userName"

try:
    session.execute(drop_table_if_exists)
except Exception as e:
    print(e)              

### Close the session and cluster connection¶

In [406]:
session.shutdown()
cluster.shutdown()